This colab notebook is based on the tutorial for semantic image segmentation of images containing people. It involves U-net architecture.

The original paper on U-net architecture https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/

The link for youtube video I referred for the same - https://youtu.be/azM57JuQpQI

In [ ]:
from re import I
import tensorflow as tf
import os
import numpy as np
import random
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt

Image parameters and dataset path

In [ ]:
dir_path_img = "./../Downloads/Human-Segmentation-Dataset-master/Training_Images/"
images = next(os.walk(dir_path_img))[2]
dir_path_msk = "./../Downloads/Human-Segmentation-Dataset-master/Ground_Truth/"
masks = next(os.walk(dir_path_msk))[2]

# image properties
img_width = 128
img_height = 128
img_channels = 3

Resizing the images and preparing the sample input and output

In [ ]:
imgs = np.zeros((len(images), img_height, img_width, img_channels), dtype = np.uint8)
msks = np.zeros((len(images), img_height, img_width, 1), dtype = bool)

# image resizing
for i in range(1, len(images) + 1):
    s = str(i)
    img = imread(dir_path_img + s + ".jpg")
    img = resize(img, (img_height, img_width, img_channels), mode = 'constant', preserve_range = True)
    imgs[i - 1] = img
    msk = np.zeros((img_height, img_width, 1), dtype = bool)
    msk1 = imread(dir_path_msk + s + ".png")
    msk1 = np.expand_dims(resize(msk1, (img_height, img_width), mode = 'constant', preserve_range = True), axis = -1)
    msks[i - 1] = np.maximum(msk, msk1)

The U-net architecture

In [ ]:
# input layer
inp_img = tf.keras.layers.Input((img_height, img_width, img_channels))
inputs = tf.keras.layers.Lambda(lambda x: x/255)(inp_img)

# convolutional and max pooling layers of contraction path
cl1 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(inputs)
cl1 = tf.keras.layers.Dropout(0.1)(cl1)
cl1 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl1)
pl1 = tf.keras.layers.MaxPooling2D((2,2))(cl1)

cl2 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(pl1)
cl2 = tf.keras.layers.Dropout(0.1)(cl2)
cl2 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl2)
pl2 = tf.keras.layers.MaxPooling2D((2,2))(cl2)

cl3 = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(pl2)
cl3 = tf.keras.layers.Dropout(0.2)(cl3)
cl3 = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl3)
pl3 = tf.keras.layers.MaxPooling2D((2,2))(cl3)

cl4 = tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(pl3)
cl4 = tf.keras.layers.Dropout(0.2)(cl4)
cl4 = tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl4)
pl4 = tf.keras.layers.MaxPooling2D((2,2))(cl4)

cl5 = tf.keras.layers.Conv2D(256, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(pl4)
cl5 = tf.keras.layers.Dropout(0.3)(cl5)
cl5 = tf.keras.layers.Conv2D(256, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl5)

# expansive path
ul6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides = (2,2), padding = 'same')(cl5)
ul6 = tf.keras.layers.concatenate([ul6,cl4])
cl6 = tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(ul6)
cl6 = tf.keras.layers.Dropout(0.2)(cl6)
cl6 = tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl6)

ul7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides = (2,2), padding = 'same')(cl6)
ul7 = tf.keras.layers.concatenate([ul7,cl3])
cl7 = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(ul7)
cl7 = tf.keras.layers.Dropout(0.2)(cl7)
cl7 = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl7)

ul8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides = (2,2), padding = 'same')(cl7)
ul8 = tf.keras.layers.concatenate([ul8,cl2])
cl8 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(ul8)
cl8 = tf.keras.layers.Dropout(0.1)(cl8)
cl8 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl8)

ul9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides = (2,2), padding = 'same')(cl8)
ul9 = tf.keras.layers.concatenate([ul9,cl1])
cl9 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(ul9)
cl9 = tf.keras.layers.Dropout(0.1)(cl9)
cl9 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(cl9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation = 'sigmoid')(cl9)
model = tf.keras.Model(inputs = [inputs], outputs = [outputs])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
# model checkpin
cp = tf.keras.callbacks.ModelCheckpoint('img_seg.h5', verbose = 1, save_best_only = True)
# callbacks
callbacks = [
    #tf.keras.callbacks.EarlyStopping(patience = 2, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = 'logs')
]

In [ ]:
results = model.fit(imgs, msks, validation_split = 0.1, batch_size = 16, epochs = 100, callbacks = callbacks)